In [ ]:
!pip install -U -q google-generativeai

In [ ]:
import os
import time
import google.generativeai as genai
import IPython.display
from IPython.display import Markdown

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
def upload_to_gemini(path, mime_type=None):
  """Uploads a file to Gemini.
  See
  https://ai.google.dev/gemini-api/docs/prompting_with_media
  https://ai.google.dev/gemini-api/docs/document-processing?lang=python
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"upload file'{file.display_name}'as:{file.uri}")
  return file

def wait_for_files_active(files):
  print("Waiting for file processing...")
  for name in (file.name for file in files):
    file = genai.get_file(name)
    while file.state.name == "PROCESSING":
      print(".", end ="", flush=True)
      time.sleep(10)
      file = genai.get_file(name)
    if file.state.name !="ACTIVE":
      raise Exception(f"File {file.name} failed to process")
  print("...all file ready")
  print()

  """
  Gemini 1.5 Pro ve 1.5 Flash,en fazla 3.600 belge sayfasını destekler. Doküman sayfaları aşağıdaki metin veri MIME türlerinden birinde olmalıdır:

PDF - application/pdf
JavaScript: application/x-javascript, text/javascript
Python: application/x-python, text/x-python
TXT - text/plain
HTML - text/html
CSS - text/css
Markdown - text/md
CSV - text/csv
XML - text/xml
RTF - text/rtf
Her belge sayfası 258 jetona eşittir.
"""

In [ ]:
system_instruction = """
Sana 2 dosya vereceğim : İçerik ve Sorular dosyası.
1. İçerik dosyasını incele.
2. Sorular dosyasındaki sorulara teker teker cevap ver.
3. İçerikte yeterli bilgi yoksa bilginin içerik dosyasında olmadığını yaz.
4. Sorular yorum sorusu ise açık ve anlaşılır yorumları gerekli argümanlarla destekleyerek yaz.
5. Cevapşarın 3-4 cümleyi geçmesin. Kısa ve net yaz."""

generation_config ={
    "temperature" : 0.4,
    "top_p" : 0.95,
    "top_k" : 64,
    "max_output_tokens" : 8192,
    "response_mime_type" : "text/plain", # This is the likely cause of the error

}

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
    system_instruction=system_instruction,
)

In [ ]:
files = [
  upload_to_gemini("Sorular.txt", mime_type= "text/plain"),
  upload_to_gemini("makale.pdf", mime_type= "application/pdf"),
    ]

wait_for_files_active(files)

upload file'Sorular.txt'as:https://generativelanguage.googleapis.com/v1beta/files/xx4ag9l0jvhm
upload file'makale.pdf'as:https://generativelanguage.googleapis.com/v1beta/files/vnvvls4cy37s
Waiting for file processing...
...all file ready



In [ ]:
chat_sesion = model.start_chat(
    history=[
        {
            "role": "user",
            "parts": [
                files[0],
                files[1],
                "İçerik dosyasını ve sorular dosyasını incele"
            ],
        },
    ]
)

In [ ]:
print(model.count_tokens(chat_sesion.history))

total_tokens: 5290



In [ ]:
chat_sesion

ChatSession(
    model=genai.GenerativeModel(
        model_name='models/gemini-1.5-flash',
        generation_config={'temperature': 0.4, 'top_p': 0.95, 'top_k': 64, 'max_output_tokens': 8192, 'response_mime_type': 'text/plain'},
        safety_settings={},
        tools=None,
        system_instruction='\nSana 2 dosya vereceğim : İçerik ve Sorular dosyası.\n1. İçerik dosyasını incele.\n2. Sorular dosyasındaki sorulara teker teker cevap ver.\n3. İçerikte yeterli bilgi yoksa bilginin içerik dosyasında olmadığını yaz.\n4. Sorular yorum sorusu ise açık ve anlaşılır yorumları gerekli argümanlarla destekleyerek yaz.\n5. Cevapşarın 3-4 cümleyi geçmesin. Kısa ve net yaz.',
        cached_content=None
    ),
    history=[protos.Content({'parts': [{'file_data': {'file_uri': 'https://gene.../xx4ag9l0jvhm', 'mime_type': 'text/plain'}}, {'file_data': {'file_uri': 'https://gene.../vnvvls4cy37s', 'mime_type': 'application/pdf'}}, {'text': 'İçerik dosya...yasını incele'}], 'role': 'user'})]
)

In [ ]:
response = chat_sesion.send_message("Sana verilen dosyadaki her bir soruyu ve cevabını yaz:")
Markdown(response.text)

İşte soruların ve cevaplarının listesi:

**1. Makaleyi özetle**

Makale, görüntü analizinde kullanılan derin öğrenme modelleri ve bunların mimarisi hakkında bilgi veriyor. Klasik makine öğrenmesi yöntemlerinin aksine, derin öğrenme ham veriler üzerinde çalışarak özellik vektörü çıkarma ihtiyacını ortadan kaldırıyor.  Makalede, evrişimli sinir ağları (ESA) mimarisi ve AlexNet, ZFNet, GoogLeNet, Microsoft ResNet ve R-CNN gibi temel mimariler ayrıntılı olarak açıklanıyor.

**2. Makalenin yazarları kimlerdir?**

Makalenin yazarları Özkan İNİK ve Erkan ÜLKER'dir.

**3. Makalenin başlığı nedir?**

Makalenin başlığı "Derin Öğrenme ve Görüntü Analizinde Kullanılan Derin Öğrenme Modelleri" dir.

**4. Makale nerede yayınlanmıştır?**

Makale Gaziosmanpaşa Bilimsel Araştırma Dergisi'nde (GBAD) yayınlanmıştır.

**5. Kaç referans kullanılmıştır?**

İçerik dosyasında referans sayısı belirtilmemiş.  Bu bilgi içerik dosyasında yok.

In [ ]:
print(response.usage_metadata)

prompt_token_count: 5356
candidates_token_count: 281
total_token_count: 5637



In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.9 MB/s eta 0:00:00


In [ ]:
import gradio as gr

In [ ]:
def generate_response(prompt, history):
  response = chat_sesion.send_message(prompt)
  return response.text

In [ ]:
gr.ChatInterface(generate_response, type = "messages" ).launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ee6259f39a45ebb67f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
